In [50]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from scipy.special import softmax



# 생선분류 데이터셋 정보확인
file_name = "./data/05_생선_분류_데이터셋.csv"
fish = pd.read_csv(file_name)
fish

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...
154,Smelt,12.2,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,14.3,15.2,2.8728,2.0672


In [3]:
fish.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Species   159 non-null    object 
 1   Weight    159 non-null    float64
 2   Length    159 non-null    float64
 3   Diagonal  159 non-null    float64
 4   Height    159 non-null    float64
 5   Width     159 non-null    float64
dtypes: float64(5), object(1)
memory usage: 7.6+ KB


In [4]:
fish.describe()

,Weight,Length,Diagonal,Height,Width
count,159.000000,159.000000,159.000000,159.000000,159.000000
mean,398.326415,28.415723,31.227044,8.970994,4.417486
std,357.978317,10.716328,11.610246,4.286208,1.685804
min,0.000000,8.400000,8.800000,1.728400,1.047600
25%,120.000000,21.000000,23.150000,5.944800,3.385650
50%,273.000000,27.300000,29.400000,7.786000,4.248500
75%,650.000000,35.500000,39.650000,12.365900,5.584500
max,1650.000000,63.400000,68.000000,18.957000,8.142000


In [6]:
# Species : 종류
# weight : 무게
# length : 길이
# Diagonal : 대각선 길이
# height : 높이
# width : 두께

# 분류는 오브젝트 타입으로
# 종류 조회
fish.Species.unique()

array(['Bream', 'Roach', 'Whitefish', 'Parkki', 'Perch', 'Pike', 'Smelt'],
      dtype=object)

In [10]:
# 훈련데이터(input, target) 생성
# 변수 : fish_input, fish_target
fish_input = fish[["Weight","Length","Diagonal","Height","Width"]].to_numpy() # 2차원
fish_target = fish["Species"].to_numpy() # 1차원
print(fish_input.shape,fish_target.shape)

(159, 5) (159,)


In [14]:
# 전처리 끝났다 가정
# 훈련데이터와 테스트데이터로 분리
# 데이터 셋 분류
train_input, test_input, train_target, test_target = train_test_split(fish_input,fish_target,
                                                                      test_size=0.25,random_state=42)
print(train_input.shape,train_target.shape)
print(test_input.shape,test_target.shape)

(119, 5) (119,)
(40, 5) (40,)


In [17]:
# 정규화(표준화) 처리 : 다양한 단위의 범위를 일괄적인 범위로 맞추기 위해 standard 사용
# train_scaled, test_scaled
# 객체생성
ss = StandardScaler()
# 훈련시키기 - 정규화 데이터 생성
ss.fit(train_input)
# 생성된 정규화 데이터로 변환
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

print(train_scaled.shape,test_scaled.shape)

(119, 5) (40, 5)


In [22]:
# 훈련 / 회귀? 분류? -> 종속변수를 기준(문자 -> 분류)
# KNN으로 확률적 분류 / 훈련/평가 하기

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled,train_target)

print(kn.score(train_scaled,train_target))
print(kn.score(test_scaled,test_target))

# 해석
# 정상이긴 하지만 과대적합의심 가능(둘 사이가 0.4 차이남)

0.8907563025210085
0.85


In [23]:
# 종속변수의 값의 순서 확인
fish.Species.unique()

array(['Bream', 'Roach', 'Whitefish', 'Parkki', 'Perch', 'Pike', 'Smelt'],
      dtype=object)

In [24]:
# 모델이 가지고 있는 종속변수 순서 확인 -> 오름차순
kn.classes_

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [26]:
# 테스트데이터에서 상위 5개 추출
test_scaled[:5]

array([[-0.88741352, -0.91804565, -1.03098914, -0.90464451, -0.80762518],
       [-1.06924656, -1.50842035, -1.54345461, -1.58849582, -1.93803151],
       [-0.54401367,  0.35641402,  0.30663259, -0.8135697 , -0.65388895],
       [-0.34698097, -0.23396068, -0.22320459, -0.11905019, -0.12233464],
       [-0.68475132, -0.51509149, -0.58801052, -0.8998784 , -0.50124996]])

In [27]:
# 5개 테스트데이터로 예측
kn.predict(test_scaled[:5])

array(['Perch', 'Smelt', 'Pike', 'Perch', 'Perch'], dtype=object)

In [28]:
test_target[:5]

array(['Perch', 'Smelt', 'Pike', 'Whitefish', 'Perch'], dtype=object)

In [29]:
# 비교하면서 뽑기
np.column_stack((test_target[:5],kn.predict(test_scaled[:5])))

array([['Perch', 'Perch'],
       ['Smelt', 'Smelt'],
       ['Pike', 'Pike'],
       ['Whitefish', 'Perch'],
       ['Perch', 'Perch']], dtype=object)

In [31]:
## 5개 데이터에 대해 확률적으로 계산된 값 확인 -> 확률적으로 계산되고 있다
proba = kn.predict_proba(test_scaled[:5])
np.round(proba,decimals=4)

array([[0.    , 0.    , 1.    , 0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 1.    , 0.    ],
       [0.    , 0.    , 0.    , 1.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.6667, 0.    , 0.3333, 0.    , 0.    ],
       [0.    , 0.    , 0.6667, 0.    , 0.3333, 0.    , 0.    ]])

In [ ]:
# KNN은 실무에서 잘 안쓴다 -> 한계존재 / 근사치는 잘 확인하지만 거리에 따라 한계존재로 오차 커짐 

In [ ]:
# 로지스틱회귀 분류모델(Logistic Regression)
"""
<분류모델은 이진분류와 다중분류로 구분됨>
- 이진분류 : 둘중 한개 선택, 시그모이드(sigmoid) 알고리즘이 내부적으로 사용됨
(시그모이드 : 0과 1로 구분하는 알고리즘)
- 다중분류 : 3개이상 중 선택, 소프트맥스(softmax) 알고리즘이 내부적으로 사용됨
(소프트맥스 : 여러새 중 확률적으로 구분하는 알고리즘)
"""



In [38]:
# 로지스틱회귀 분류모델(Logistic Regression) - 이진분류
# 이진분류를 위한 데이터로 재구성하기 : Bream과 Smelt값 사용
# Bream과 Smelt값 인덱스 위치 확인하기
bream_smelt_indexes = (train_target == "Bream") | (train_target == "Smelt")
print(bream_smelt_indexes)

# 훈련에 사용할 독립변수와 종속변수 데이터 추출
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]
print(train_bream_smelt.shape,target_bream_smelt.shape)

[ True False  True False False False False  True False False False  True
 False False False  True  True False False  True False  True False False
 False  True False False  True False False False False  True False False
  True  True False False False False False  True False False False False
 False  True False  True False False  True False False False  True False
 False False False False False  True False  True False False False False
 False False False False False  True False  True False False  True  True
 False False False  True False False False False False  True False False
 False  True False  True False False  True  True False False False False
 False False False False  True  True False False  True False False]
(33, 5) (33,)


In [40]:
# 훈련하기
lr = LogisticRegression()
lr.fit(train_bream_smelt,target_bream_smelt)
lr.score(train_bream_smelt,target_bream_smelt)

1.0

In [41]:
# 상위 5개만 예측하기, 종속변수들에 대한 확률값 확인
# 0번쨰 ; Bream, 1번째 : Smelt
lr.predict_proba(train_bream_smelt[:5])

array([[0.99759855, 0.00240145],
       [0.02735183, 0.97264817],
       [0.99486072, 0.00513928],
       [0.98584202, 0.01415798],
       [0.99767269, 0.00232731]])

In [42]:
# 종속변수의 순서 확인
lr.classes_

array(['Bream', 'Smelt'], dtype=object)

In [43]:
# 시그모이드 결과값 확인
lr.decision_function(train_bream_smelt[:5])
# 음수 : 왼쪽편, 양수 : 오른쪽편
# 0보다 작으면 0, 0보다 크면 1

array([-6.02927744,  3.57123907, -5.26568906, -4.24321775, -6.0607117 ])

In [45]:
# 로지스틱회귀 분류모델(Logistic Regression) - 다중분류
# 종속변수 값 다 쓰면 된다
# 훈련하기
# 하이퍼파라미터 삽입
"""
- C : 규제강도 조절값(릿지, 라쏘 모델의 alpha와 동일) / alpha값의 경우 클수록 강도가 커짐
     / C의 값은 작을 수록 강도가 커짐 - 기본 값 : 1 -> 기본 값보다 큰 값을 사용했기에 강도 완화
    
- max_iter : 훈련 반복횟수 / 기본 값 100 / 훈련 중 반복횟수를 초과하여 훈련을 해야하는 경우(모델이 알아서)
            경고창이 뜨는 경우 있다. 그래서 반복횟수를 크게 잡아도 된다.
"""
lr = LogisticRegression(C=20,max_iter=1000)
lr.fit(train_scaled,train_target)
print(lr.score(train_scaled,train_target))
print(lr.score(test_scaled,test_target))

0.9327731092436975
0.925


In [46]:
# 상위 5개 데이터로 예측
lr.predict(test_scaled[:5])

array(['Perch', 'Smelt', 'Pike', 'Roach', 'Perch'], dtype=object)

In [49]:
# 확률값 확인
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba,decimals=3))
print(lr.classes_)

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [51]:
# 소프트맥스 알고리즘을 이용해 확률값 확인
decision = lr.decision_function(test_scaled[:5])
proba = softmax(decision,axis=1)
print(np.round(proba,decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
